In [41]:
run instagram_poster

In [42]:
import re
import yaml
from datetime import date
from markdown2 import markdown
from mdplain import plain
from pathlib import Path

places = []
taste_labels = ["Do Not Recommend", "Something Going For It", "Good", "Phenomenal"]

def rating_to_formatting(rating, rating_labels):
    return rating_labels[rating]
def rating_html(rating, rating_labels):
    return "&nbsp;".join(
        [
            f'<span style="color: {color if rating == ix else faded_color}" aria-hidden="{"false" if rating == ix else "true"}">{label}</span>'
            for ix, (label, color) in enumerate(zip(rating_labels, rating_colors))
        ]
    )
def suffix(d):
    return "th" if 11 <= d <= 13 else {1: "st", 2: "nd", 3: "rd"}.get(d % 10, "th")
def custom_strftime(format_, t):
    return t.strftime(format_).replace("{S}", str(t.day) + suffix(t.day))
def format_visited(visited):
    return custom_strftime("{S} %B %Y", visited)
def format_blurb(md):
    return " ".join(plain(re.sub(r"\s+", " ", md.strip())).split(" ")[:50]) + "..."
    
for place_md in Path("../places").glob("*.md"):
    try: 
        slug = place_md.parts[-1][:-3]
        relative_url = f"/places/{slug}/"
        with open(place_md) as f:
            _, frontmatter, md = f.read().split("---", 2)
        meta = yaml.load(frontmatter, Loader=yaml.Loader)
        meta["url"] = relative_url
        meta["slug"] = slug
        visited = date.fromisoformat(meta["visited"])
        meta["visited_display"] = format_visited(visited)
        meta["review_age"] = (date.today() - visited).days
        meta["taste_label"] = rating_to_formatting(meta["taste"], taste_labels)
        meta["blurb"] = format_blurb(md)

        places.append(meta)

    except Exception as e:
        print(place_md.name, e)


In [59]:
for place_md in Path("../places").glob("*.md"):

    with open(place_md, 'r') as file:
        content = file.read()
    
    # Split content to find the end of the YAML front matter
    parts = content.split('---', 2)
    if len(parts) > 2:
        # Insert the new field after the front matter
        new_content = parts[0] + '---' + parts[1] + 'instagram_published: False\n' + '---' + parts[2]
    else:
        new_content = content  # Use original content if the split didn't work as expected
    
    # Write the updated content back to the same file
    with open(place_md, 'w') as file:
        file.write(new_content)

In [56]:
print(new_content)

---
name: Beit Rima
cuisine: Middle Eastern
drinks: True
visited: "2022-08-20"
address: 86 Carl St
area: Haight-Ashbury
taste: 2
value: 2
lat: 37.7659340593549
lon: -122.44968514756269
menu: http://beitrimasf.com/menu/
phone: "+14155661274"
instagram_published: False
---

Really good Middle Eastern food. We got a literal feast, and a bottle of Palestinian wine. Between 4 of us it costed $35 a head including tax and tip. Good value for what we got. 

Stand-outs were the **batata harra**, **house pita with spices**, and **hamleh** (fresh chickpeas that are charred in their shells). Although everything else we got was also good including the falafel, muhamara, baba ghanoush and ful.



In [ ]:
def add_instagram_field_to_md_file(file_path):
    # Read the existing content from the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Split content to find the end of the YAML front matter
    parts = content.split('---', 2)
    if len(parts) > 2:
        # Insert the new field after the front matter
        new_content = parts[0] + '---' + parts[1] + '\ninstagram_published: false\n' + '---' + parts[2]
    else:
        new_content = content  # Use original content if the split didn't work as expected

    # Write the updated content back to the same file
    with open(file_path, 'w') as file:
        file.write(new_content)

# Specify the path to your Markdown file
file_path = 'review.md'

# Call the function to update the file
add_instagram_field_to_md_file(file_path)

print("The file has been updated successfully.")

In [7]:
place = places[83]

In [43]:
credentials_file_path = 'credentials.json'
bot = InstagramBot(credentials_file_path)
bot.login()
bot.upload_post(place)
bot.close_driver()
